In [1]:
import pandas as pd
import ttach as tta

In [2]:
# !pip install optuna

In [3]:
import os
import shutil
import pathlib
import gc
from pathlib import Path
from PIL import Image
import pandas as pd
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp

In [6]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [7]:
seed_everything(42)

In [8]:
target = "1"
# NTB = 7

folder = "train"  # "train"
if folder == "validation":
    target = "1"
    
NTB = 5 # 3

In [9]:

CFGS = [    
    {
        'model_name': 'Unet',
        'backbone': 'timm-efficientnet-b8',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '../../output/exp_s1_01/Unet/timm-efficientnet-b8/checkpoint_dice.pth',
        'threshold': 0.3,
        'tta': True
    }, 
    
    {
    'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-512/checkpoint_dice.pth',
        'threshold': 0.1,
        'tta': True
    }, 
    {
    'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [384, 384],
        'num_classes': 1,
        'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-384/checkpoint_dice.pth',
        'threshold': 0.16,
        'tta': True
    }, 
    {
    'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-256-100-epochs/checkpoint_dice.pth',
        'threshold': 0.04,
        'tta': True
    },
    {
    'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-256-bce-train_2/checkpoint_dice.pth',
        'threshold': 0.32,
        'tta': False
    },
    {
    'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-256-train_2/checkpoint_dice.pth',
        'threshold': 0.32,
        'tta': False
    },
    
    
]

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [11]:
data_transforms = {
    "train": A.Compose([
        A.Resize(*CFGS[0]['img_size'], interpolation=cv2.INTER_NEAREST),
        ], p=1.0),
    
    "valid": A.Compose([
        A.Resize(*CFGS[0]['img_size'], interpolation=cv2.INTER_NEAREST),
        ], p=1.0),
    
    
    "test": A.Compose([
            A.Resize(*CFGS[0]['img_size'], interpolation=cv2.INTER_NEAREST),
    ], p=1.0),
    
}

In [12]:
class ContrailDataset:
    def __init__(self, df, transform=None):
        self.df = df  
        self.images = df['image']
        self.transform =transform
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
#         print("../../input" + self.images[idx])
        image = np.load("../../input" + self.images[idx]).astype(float)   
        
        if self.transform :
            data = self.transform(image=image)
            image  = data['image']
            image = np.transpose(image, (2, 0, 1))
        return torch.tensor(image)

In [13]:
class Net(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        self.model = smp.Unet(
            encoder_name=cfg["backbone"],      # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights=None,     # use `imagenet` pre-trained weights for encoder initialization
            in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=cfg["num_classes"],        # model output channels (number of classes in your dataset)
            activation=None
        )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask

In [14]:
# params = {
#  'w1': 0.951382694462493,
#  'w2': 0.36024417206973613,
#  'w3': 0.16498218172962048,
#  'w4': 0.6047624543660389,
#  'w5': 0.5106552868387371,
#  'w6': 0.33710127631320935
# }

# params = {
#  'w1': 0.33623293716110597,
#  'w2': 0.9737207909271809,
#  'w3': 0.3083112197058403,
#  'w4': 0.4905850723996572,
#  'w5': 0.19376624099524614,
#  'w6': 0.7643271123289296,
# }

params = {
    'w1': 0.5294649933133848,
    'w2': 0.9807394388336473,
    'w3': 0.4183536450466668,
    'w4': 0.13610731095623466,
    'w5': 0.7324196781702746,
    'w6': 0.6022020327386348
}

threshold = 0.14

def weighted_ensemble(params, final_preds):    
    for index, val in enumerate(params.keys()):
        if index == 0:
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    return preds


def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform

In [16]:
if target == "0":
    val_df = pd.read_csv(f'../../input/pseudo/{folder}_data_empty_{NTB}.csv') 
else:
    val_df = pd.read_csv(f'../../input/pseudo/{folder}_data_{NTB}.csv') 

    
final_preds = []

for idx, cfg in enumerate(CFGS):    
    val_df['id'] = val_df['label'].apply(lambda x: x.split('/')[3])

    val_transform = data_transforms["test"]
    valid_dataset = ContrailDataset(val_df, transform=get_transform(cfg['img_size']))  

    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 64, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )
    
    
    print(cfg)
#     model = Net(cfg['backbone'], cfg['num_classes'])
    model = Net(cfg) 
    model = torch.nn.DataParallel(model).cuda()
    
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    if cfg['tta']:
        model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')

    model.to(device)
    model.eval()
    preds = []
    for idx, image in enumerate(tqdm(valid_loader)):  
        image = image.to(device, dtype=torch.float)    
        with torch.inference_mode():
            pred = model(image)
            if cfg['img_size'][0] != 256:
                pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest') 
            preds.append(pred) 
    final_preds.append(torch.cat(preds))
    
    
    del model, preds
    torch.cuda.empty_cache()
    gc.collect()
    
    
final_preds = weighted_ensemble(params, final_preds)
final_preds = (nn.Sigmoid()(final_preds)>threshold).double()

{'model_name': 'Unet', 'backbone': 'timm-efficientnet-b8', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '../../output/exp_s1_01/Unet/timm-efficientnet-b8/checkpoint_dice.pth', 'threshold': 0.3, 'tta': True}


  0%|          | 0/145 [00:00<?, ?it/s]

{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '../../output/contrail_exp_b7/efficientnet-b7-512/checkpoint_dice.pth', 'threshold': 0.1, 'tta': True}


  0%|          | 0/145 [00:00<?, ?it/s]

{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [384, 384], 'num_classes': 1, 'model_pth': '../../output/contrail_exp_b7/efficientnet-b7-384/checkpoint_dice.pth', 'threshold': 0.16, 'tta': True}


...

  0%|          | 0/145 [00:00<?, ?it/s]

{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '../../output/contrail_exp_b7/efficientnet-b7-256-bce-train_2/checkpoint_dice.pth', 'threshold': 0.32, 'tta': False}


  0%|          | 0/145 [00:00<?, ?it/s]

{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '../../output/contrail_exp_b7/efficientnet-b7-256-train_2/checkpoint_dice.pth', 'threshold': 0.32, 'tta': False}


  0%|          | 0/145 [00:00<?, ?it/s]

In [17]:
ids = val_df['id'].values
for (val, label_id) in tqdm(zip(final_preds, ids), total=len(ids)): 
    mask = val.view(256, 256, 1).detach().cpu().numpy()
    
    if target == "0":
        np.save(f"../../input/pseudo/{folder}_data_empty_{NTB}/{label_id}/label.npy", mask.astype('float16')) 
    else:
        np.save(f"../../input/pseudo/{folder}_data_{NTB}/{label_id}/label.npy", mask.astype('float16')) 



  0%|          | 0/9259 [00:00<?, ?it/s]

In [18]:
# val_df.image.values[0]

In [19]:
val_df.shape

(9259, 3)